In [1]:
import cirq
import numpy as np
import matplotlib.pyplot as plt
from qiskit_ibm_provider import IBMProvider
from qiskit import QuantumCircuit, transpile
from collections import Counter
from mitiq.rem import generate_tensored_inverse_confusion_matrix
from mitiq.rem.inverse_confusion_matrix import mitigate_measurements
from mitiq.typing import MeasurementResult

In [2]:
provider = IBMProvider()
backend = provider.get_backend('ibm_kyoto')

<h2>One qubit:</h2>

In [12]:
layout = [20]
repcnt = 8192

In [8]:
q0 = cirq.LineQubit(0)
circs = [cirq.Circuit(),cirq.Circuit(cirq.Circuit(cirq.X(q0)))]
for circ in circs:
    circ.append(cirq.measure(q0,key='answer'))

In [11]:
circs_qiskit = []
for circ in circs:
    circuit_qiskit = QuantumCircuit.from_qasm_str(circ.to_qasm())
    transpiled_circuit = transpile(circuit_qiskit,backend=backend,optimization_level=0,initial_layout=layout)
    circs_qiskit.append(transpiled_circuit)

In [23]:
with open('./Jobs/2.txt','x') as job_id_file:
        job = backend.run(circs_qiskit,shots=repcnt)   #Run job
        job_id_file.write(job.job_id()+'\n')    #Write to file
        print('Submitted job with job id '+job.job_id())

print('All complete.')

Submitted job with job id cnvh9cdgpnkg008f05fg
All complete.


In [17]:
def confusion_matrix(job_id,num_qubits,shots):
    counts = None
    job = provider.backend.retrieve_job(job_id.strip())
    if not job.done():
        print(job_id.strip()+' is not complete') # to avoid silent fail
        return counts
    results = job.result()
    counts = results.get_counts()
    res = np.zeros(shape=(2**num_qubits,2**num_qubits)) #(type,prob,scale_factor)
    for i in range(2**num_qubits):
        cnt = counts[i] #prepared state i
        for j in range(2**num_qubits): #measured state j
            #res[i,j] = meas i prep j
            res[j,i] = cnt.get(bin(j)[2:].zfill(num_qubits),0)/shots
    return res

In [57]:
job_id = 'cnvh9cdgpnkg008f05fg'
A = confusion_matrix(job_id,1,repcnt)

In [58]:
A_pinv = generate_tensored_inverse_confusion_matrix(1,[A])

In [48]:
rae = 1.020e-2
B = np.array([[1-rae,rae],[rae,1-rae]])
Bi = np.linalg.inv(B)

In [56]:
print(A)
print(Bi@A)
print(A_pinv)

[[0.97827148 0.01416016]
 [0.02172852 0.98583984]]
[[0.98823141 0.00404263]
 [0.01176859 0.99595737]]
[[ 1.02253735 -0.01468726]
 [-0.02253735  1.01468726]]


In [59]:
print(A)
print(Bi@A)
print(A_pinv)

[[0.97338867 0.01428223]
 [0.02661133 0.98571777]]
[[0.98324691 0.00416724]
 [0.01675309 0.99583276]]
[[ 1.02774596 -0.01489118]
 [-0.02774596  1.01489118]]
